In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

train_df = pd.read_excel('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/WikiBias/mitigation/train.xlsx')
val_df = pd.read_excel('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/WikiBias/mitigation/dev.xlsx')
test_df = pd.read_excel('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/WikiBias/mitigation/test.xlsx')

In [ ]:
print("Train set size: ", train_df.shape)
print("Val set size: ", val_df.shape)
print("Test set size: ", test_df.shape)

Train set size:  (2117, 4)
Val set size:  (431, 4)
Test set size:  (852, 4)


In [ ]:
# train_df = pd.read_excel('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/WikiBias/detection/train.xlsx')
# val_df = pd.read_excel('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/WikiBias/detection/dev.xlsx')
# test_df = pd.read_excel('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/WikiBias/detection/test.xlsx')

In [ ]:
# print("Train set size: ", train_df.shape)
# print("Val set size: ", val_df.shape)
# print("Test set size: ", test_df.shape)

Train set size:  (5028, 3)
Val set size:  (1066, 3)
Test set size:  (2104, 3)


# Fine-Tuning Phi-3.5 3.8B

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.3: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Prep

In [ ]:
# Define a function to create a conversational format
def format_row(row):
    return [
        {"from": "system", "value": "You are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
        {"from": "human", "value": "The biased sentence is: " + row["biased_sent"]},
        {"from": "assistant", "value": row["debiased_sent"]}
    ]

train_conversations = pd.DataFrame()
val_conversations = pd.DataFrame()
test_conversations = pd.DataFrame()

train_conversations["conversations"] = train_df.apply(format_row, axis=1)
val_conversations["conversations"] = val_df.apply(format_row, axis=1)
test_conversations["conversations"] = test_df.apply(format_row, axis=1)


In [ ]:
from datasets import Dataset
train_ds = Dataset.from_pandas(train_conversations)
val_ds = Dataset.from_pandas(val_conversations)
test_ds = Dataset.from_pandas(test_conversations)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "assistant", "system" : "system"}
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

train_dataset = train_ds.map(formatting_prompts_func, batched = True,)
val_dataset = val_ds.map(formatting_prompts_func, batched = True,)
test_dataset = test_ds.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/2117 [00:00<?, ? examples/s]

Map:   0%|          | 0/431 [00:00<?, ? examples/s]

Map:   0%|          | 0/852 [00:00<?, ? examples/s]

In [ ]:
train_dataset["conversations"][4]

[{'from': 'system',
  'value': "You are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
 {'from': 'human',
  'value': 'The biased sentence is: It asks such questions as : How should men interact in society ?'},
 {'from': 'assistant',
  'value': 'It asks such questions as : How should people interact in society ?'}]

## Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3,
        evaluation_strategy="steps",
        eval_steps=10,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        load_best_model_at_end = True
    ),
     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/2117 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/2117 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/2117 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/2117 [00:00<?, ? examples/s]

Converting eval dataset to ChatML (num_proc=2):   0%|          | 0/431 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=2):   0%|          | 0/431 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/431 [00:00<?, ? examples/s]

Truncating eval dataset (num_proc=2):   0%|          | 0/431 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,117 | Num Epochs = 3 | Total steps = 792
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416/2,039,024,640 (1.47% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
10,1.637200,1.491584
20,0.737900,0.713933
30,0.776100,0.657782
40,0.792000,0.650499
50,0.546400,0.645678
60,0.567200,0.641650
70,0.750200,0.638736


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


KeyboardInterrupt: 

## Qualitative Eval - Inference

In [ ]:
test_df

,biased_sent,bias_tags_biased_sent,debiased_sent,bias_tags_debiased_sent
0,Target also offers the dog as a stuffed toy fo...,['member'],Target also offers the dog as a stuffed toy fo...,[]
1,A contract dispute with the Dolphins led him t...,[],A contract dispute with the Dolphins led him t...,[]
2,Gring was a relative of numerous descendants o...,[],Gring was a relative of numerous descendants o...,[]
3,"In 1996 , Edwards was appointed to the Tongan ...",['Majesty'],"In 1996 , Edwards was appointed to the Tongan ...",[]
4,The Pinkston blocks are to receive funding and...,"['most', 'residents', 'agree', 'that', 'bulldo...",The Pinkston blocks are to receive funding and...,[]
...,...,...,...,...
847,"The line "" Heimat bist du groer Shne "" was rep...","['not', 'fitting']","The line "" Heimat bist du groer Shne "" was rep...",[]
848,Duke University 's men 's basketball team is o...,"['is', 'the', 'most', 'hated']",Duke University 's men 's basketball team is o...,[]
849,Mitchell is most famous for her performance as...,[],Mitchell is well known for her performance as ...,[]
850,A miscarriage of justice primarily is the conv...,"['or', 'she']",A miscarriage of justice primarily is the conv...,[]


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt", "system": "system"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

index = 407
biased_sent = test_df['biased_sent'][index]

messages = [
    {"from": "system", "value": "You are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
    {"from": "human", "value": "The biased sentence is: " + biased_sent }
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
text_output = tokenizer.batch_decode(outputs)[0]
text_output = text_output.split("<|assistant|>")[1].strip()
text_output = text_output.split("<|end|>")[0].strip()
print(text_output)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Sunny Li ( ; born 4 September 1991 ) is a Chinese concert pianist .


In [ ]:
print(test_df['biased_sent'][index])

Sunny Li ( ; born 4 September 1991 ) is a exceptional young Chinese concert pianist .


In [ ]:
print(test_df['debiased_sent'][index])

Sunny Li ( ; born 4 September 1991 ) is a Chinese concert pianist .


## Evaluation

In [ ]:
def generate_debiased_sentences(biased_sent, tokenizer):

    messages = [
    {"from": "system", "value": "You are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
    {"from": "human", "value": "The biased sentence is: " + biased_sent }
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
    text_output = tokenizer.batch_decode(outputs)[0]
    text_output = text_output.split("<|assistant|>")[1].strip()
    text_output = text_output.split("<|end|>")[0].strip()
    return text_output

to_csv = { 'biased_sent': [],'debiased_sent' : [], 'model_output' : [] }

for biased_sent, debiased_sent in zip(test_df['biased_sent'], test_df['debiased_sent']):
    out_sentence = generate_debiased_sentences(biased_sent, tokenizer)
    to_csv['biased_sent'].append(biased_sent)
    to_csv['debiased_sent'].append(debiased_sent)
    to_csv['model_output'].append(out_sentence)

df = pd.DataFrame(to_csv)


In [ ]:
df

,biased_sent,debiased_sent,model_output
0,Target also offers the dog as a stuffed toy fo...,Target also offers the dog as a stuffed toy fo...,Target also offers the dog as a stuffed toy fo...
1,A contract dispute with the Dolphins led him t...,A contract dispute with the Dolphins led him t...,A contract dispute with the Dolphins led him t...
2,Gring was a relative of numerous descendants o...,Gring was a relative of numerous descendants o...,Gring was a relative of numerous descendants o...
3,"In 1996 , Edwards was appointed to the Tongan ...","In 1996 , Edwards was appointed to the Tongan ...","In 1996 , Edwards was appointed to the Tongan ..."
4,The Pinkston blocks are to receive funding and...,The Pinkston blocks are to receive funding and...,The Pinkston blocks are to receive funding and...
...,...,...,...
847,"The line "" Heimat bist du groer Shne "" was rep...","The line "" Heimat bist du groer Shne "" was rep...","The line "" Heimat bist du groer Shne "" was rep..."
848,Duke University 's men 's basketball team is o...,Duke University 's men 's basketball team is o...,Duke University 's men 's basketball team is o...
849,Mitchell is most famous for her performance as...,Mitchell is well known for her performance as ...,Mitchell is most famous for her performance as...
850,A miscarriage of justice primarily is the conv...,A miscarriage of justice primarily is the conv...,A miscarriage of justice primarily is the conv...


In [ ]:
df.to_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/wiki-phi-3.5.csv', index=False)

# Fine-Tuning Llama 3.2 3B

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.3: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Data Prep

In [ ]:
def format_row(row):
    return [
        {"from": "system", "value": "You are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
        {"from": "human", "value": "The biased sentence is: " + row["biased_sent"]},
        {"from": "assistant", "value": row["debiased_sent"]}
    ]

train_conversations = pd.DataFrame()
val_conversations = pd.DataFrame()
test_conversations = pd.DataFrame()

train_conversations["conversations"] = train_df.apply(format_row, axis=1)
val_conversations["conversations"] = val_df.apply(format_row, axis=1)
test_conversations["conversations"] = test_df.apply(format_row, axis=1)

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_conversations)
val_dataset = Dataset.from_pandas(val_conversations)
test_dataset = Dataset.from_pandas(test_conversations)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)


def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, tokenize = False, add_generation_prompt = False
        )
        for convo in convos
    ]
    return { "text" : texts, }

from unsloth.chat_templates import standardize_sharegpt
train_ds = standardize_sharegpt(train_dataset)
val_ds = standardize_sharegpt(val_dataset)
test_ds = standardize_sharegpt(test_dataset)
train_ds = train_ds.map(formatting_prompts_func, batched = True,)
val_ds = val_ds.map(formatting_prompts_func, batched = True,)
test_ds = test_ds.map(formatting_prompts_func, batched = True,)


Standardizing format:   0%|          | 0/2117 [00:00<?, ? examples/s]

Standardizing format:   0%|          | 0/431 [00:00<?, ? examples/s]

Standardizing format:   0%|          | 0/852 [00:00<?, ? examples/s]

Map:   0%|          | 0/2117 [00:00<?, ? examples/s]

Map:   0%|          | 0/431 [00:00<?, ? examples/s]

Map:   0%|          | 0/852 [00:00<?, ? examples/s]

In [ ]:
train_ds[5]["conversations"]

[{'content': "You are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.",
  'role': 'system'},
 {'content': "The biased sentence is: In 1953 , the Nehru government bowed to intense pressure from the northern Telugu-speaking districts of Madras State , and allowed them to vote to create India 's first linguistic state .",
  'role': 'user'},
 {'content': "In 1953 , the Nehru government yielded to intense pressure from the northern Telugu-speaking districts of Madras State , and allowed them to vote to create India 's first linguistic state .",
  'role': 'assistant'}]

In [ ]:
train_ds[5]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nThe biased sentence is: In 1953 , the Nehru government bowed to intense pressure from the northern Telugu-speaking districts of Madras State , and allowed them to vote to create India 's first linguistic state .<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIn 1953 , the Nehru government yielded to intense pressure from the northern Telugu-speaking districts of Madras State , and a

## Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset = val_ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        # max_steps = 60,
        evaluation_strategy="steps",
        eval_steps=10,
        # learning_rate = 2e-4,
        learning_rate = 1e-5, #decrease learning rate
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        load_best_model_at_end = True
    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/2117 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/2117 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/2117 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/2117 [00:00<?, ? examples/s]

Converting eval dataset to ChatML (num_proc=2):   0%|          | 0/431 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=2):   0%|          | 0/431 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/431 [00:00<?, ? examples/s]

Truncating eval dataset (num_proc=2):   0%|          | 0/431 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/2117 [00:00<?, ? examples/s]

Map:   0%|          | 0/431 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,117 | Num Epochs = 3 | Total steps = 792
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/1,865,526,272 (1.30% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
10,1.322100,1.172506
20,0.779600,0.958854
30,0.435700,0.755442
40,0.464800,0.601481
50,0.417000,0.509905
60,0.497900,0.473136
70,0.696600,0.454532
80,0.387300,0.440712
90,0.687800,0.430108
100,0.263800,0.422478


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


KeyboardInterrupt: 

## Qualitative Evaluation - Inference

In [ ]:
test_df

,biased_sent,bias_tags_biased_sent,debiased_sent,bias_tags_debiased_sent
0,Target also offers the dog as a stuffed toy fo...,['member'],Target also offers the dog as a stuffed toy fo...,[]
1,A contract dispute with the Dolphins led him t...,[],A contract dispute with the Dolphins led him t...,[]
2,Gring was a relative of numerous descendants o...,[],Gring was a relative of numerous descendants o...,[]
3,"In 1996 , Edwards was appointed to the Tongan ...",['Majesty'],"In 1996 , Edwards was appointed to the Tongan ...",[]
4,The Pinkston blocks are to receive funding and...,"['most', 'residents', 'agree', 'that', 'bulldo...",The Pinkston blocks are to receive funding and...,[]
...,...,...,...,...
847,"The line "" Heimat bist du groer Shne "" was rep...","['not', 'fitting']","The line "" Heimat bist du groer Shne "" was rep...",[]
848,Duke University 's men 's basketball team is o...,"['is', 'the', 'most', 'hated']",Duke University 's men 's basketball team is o...,[]
849,Mitchell is most famous for her performance as...,[],Mitchell is well known for her performance as ...,[]
850,A miscarriage of justice primarily is the conv...,"['or', 'she']",A miscarriage of justice primarily is the conv...,[]


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

index = 407
biased_sent = test_df['biased_sent'][index]

messages = [
{"role": "system", "content": "You are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
{"role": "human", "content": "The biased sentence is: " + biased_sent }
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                         temperature = 1, min_p = 0.1)
text_output = tokenizer.batch_decode(outputs)[0]
text_output = text_output.split("<|start_header_id|>assistant<|end_header_id|>\n\n")[1].strip()
text_output = text_output.split("<|eot_id|>")[0].strip()
text_output

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'Sunny Li ( ; born 4 September 1991 ) is a young Chinese concert pianist.'

In [ ]:
test_df['debiased_sent'][index]

'Sunny Li ( ; born 4 September 1991 ) is a Chinese concert pianist .'

In [ ]:
test_df['biased_sent'][index]

'Sunny Li ( ; born 4 September 1991 ) is a exceptional young Chinese concert pianist .'

## Evaluation

In [ ]:
def generate_debiased_sentences(biased_sent, tokenizer):
    tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

    messages = [
    {"role": "system", "content": "You are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
    {"role": "human", "content": "The biased sentence is: " + biased_sent }
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                            temperature = 1, min_p = 0.1)
    text_output = tokenizer.batch_decode(outputs)[0]
    text_output = text_output.split("<|start_header_id|>assistant<|end_header_id|>\n\n")[1].strip()
    text_output = text_output.split("<|eot_id|>")[0].strip()
    return text_output

to_csv = { 'biased_sent': [],'debiased_sent' : [], 'model_output' : [] }

for biased_sent, debiased_sent in zip(test_df['biased_sent'], test_df['debiased_sent']):
    out_sentence = generate_debiased_sentences(biased_sent, tokenizer)
    to_csv['biased_sent'].append(biased_sent)
    to_csv['debiased_sent'].append(debiased_sent)
    to_csv['model_output'].append(out_sentence)

df = pd.DataFrame(to_csv)



In [ ]:
df

,biased_sent,debiased_sent,model_output
0,Target also offers the dog as a stuffed toy fo...,Target also offers the dog as a stuffed toy fo...,Target offers the dog as a stuffed toy for spe...
1,A contract dispute with the Dolphins led him t...,A contract dispute with the Dolphins led him t...,A contract dispute with the Dolphins led him t...
2,Gring was a relative of numerous descendants o...,Gring was a relative of numerous descendants o...,Gring was a relative of numerous descendants o...
3,"In 1996 , Edwards was appointed to the Tongan ...","In 1996 , Edwards was appointed to the Tongan ...","In 1996, Edwards was appointed to the Tongan C..."
4,The Pinkston blocks are to receive funding and...,The Pinkston blocks are to receive funding and...,The Pinkston blocks are to receive funding and...
...,...,...,...
847,"The line "" Heimat bist du groer Shne "" was rep...","The line "" Heimat bist du groer Shne "" was rep...","The line "" Heimat bist du groer Shne "" was rep..."
848,Duke University 's men 's basketball team is o...,Duke University 's men 's basketball team is o...,The Duke University's men's basketball team is...
849,Mitchell is most famous for her performance as...,Mitchell is well known for her performance as ...,Mitchell is most famous for her performance as...
850,A miscarriage of justice primarily is the conv...,A miscarriage of justice primarily is the conv...,A miscarriage of justice primarily is the conv...


In [ ]:
df.to_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/wiki-llama-3.2-3b.csv', index=False)

# Fine-Tuning Mistral v0.3 7B

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.4: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Prep

In [ ]:
def format_row(row):
    return [
        {"from": "system", "value": "You are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
        {"from": "human", "value": "The biased sentence is: " + row["biased_sent"]},
        {"from": "assistant", "value": row["debiased_sent"]}
    ]

train_conversations = pd.DataFrame()
val_conversations = pd.DataFrame()

train_conversations["conversations"] = train_df.apply(format_row, axis=1)
val_conversations["conversations"] = val_df.apply(format_row, axis=1)

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_conversations)
val_dataset = Dataset.from_pandas(val_conversations)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "assistant", "system": "system"},
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }


train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
val_dataset = val_dataset.map(formatting_prompts_func, batched = True,)

Unsloth: Will map <|im_end|> to EOS = </s>.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Map:   0%|          | 0/2117 [00:00<?, ? examples/s]

Map:   0%|          | 0/431 [00:00<?, ? examples/s]

In [ ]:
train_dataset[5]["conversations"]

[{'from': 'system',
  'value': "You are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
 {'from': 'human',
  'value': "The biased sentence is: In 1953 , the Nehru government bowed to intense pressure from the northern Telugu-speaking districts of Madras State , and allowed them to vote to create India 's first linguistic state ."},
 {'from': 'assistant',
  'value': "In 1953 , the Nehru government yielded to intense pressure from the northern Telugu-speaking districts of Madras State , and allowed them to vote to create India 's first linguistic state ."}]

In [ ]:
print(train_dataset[5]["text"])

<|im_start|>system
You are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.<|im_end|>
<|im_start|>user
The biased sentence is: In 1953 , the Nehru government bowed to intense pressure from the northern Telugu-speaking districts of Madras State , and allowed them to vote to create India 's first linguistic state .<|im_end|>
<|im_start|>assistant
In 1953 , the Nehru government yielded to intense pressure from the northern Telugu-speaking districts of Madras State , and allowed them to vote to create India 's first linguistic state .<|im_end|>



## Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        save_on_each_node=True,
        save_steps=10,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        evaluation_strategy="steps",
        eval_steps=10,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        load_best_model_at_end = True
    ),
     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Tokenizing to ["text"] (num_proc=2):   0%|          | 0/2117 [00:00<?, ? examples/s]

Tokenizing to ["text"] (num_proc=2):   0%|          | 0/431 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,117 | Num Epochs = 1 | Total steps = 264
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/3,800,305,664 (1.10% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
10,0.594800,0.628649
20,0.595400,0.566464
30,0.620800,0.542682
40,0.665800,0.531352
50,0.471800,0.530958
60,0.469200,0.528692
70,0.611300,0.525909
80,0.541700,0.526224
90,0.592200,0.525401
100,0.413500,0.525121


Unsloth: Not an error, but MistralForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


KeyboardInterrupt: 

## Qualitative Evaluation - Inference

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "assistant", "system": "system"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference


index = 407
biased_sent = test_df['biased_sent'][index]

messages = [
    {"from": "system", "value": "You are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
    {"from": "human", "value": "The biased sentence is: " + biased_sent }
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True)
output_sentence = tokenizer.batch_decode(outputs)[0]
output_sentence

Unsloth: Will map <|im_end|> to EOS = <|im_end|>.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


"<|im_start|>system\nYou are provided with a linguistically biased sentence from a wikipedia article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained.<|im_end|>\n<|im_start|>user\nThe biased sentence is: Sunny Li ( ; born 4 September 1991 ) is a exceptional young Chinese concert pianist .<|im_end|>\n<|im_start|>assistant\nSunny Li ( ; born 4 September 1991 ) is a young Chinese concert pianist .<|im_end|>"

In [ ]:
output_sentence=output_sentence.split('<|im_start|>assistant\n')[1]
output_sentence = output_sentence.split('<|im_end|>')[0]
output_sentence

'Sunny Li ( ; born 4 September 1991 ) is a young Chinese concert pianist .'

In [ ]:
model.save_pretrained("/content/drive/Shareddrives/FYP 2024-2025/Phase-2/WikiMit/models/Mistral-v0.3-7B")
tokenizer.save_pretrained("/content/drive/Shareddrives/FYP 2024-2025/Phase-2/WikiMit/models/Mistral-v0.3-7B")

('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/WikiMit/models/Mistral-v0.3-7B/tokenizer_config.json',
 '/content/drive/Shareddrives/FYP 2024-2025/Phase-2/WikiMit/models/Mistral-v0.3-7B/special_tokens_map.json',
 '/content/drive/Shareddrives/FYP 2024-2025/Phase-2/WikiMit/models/Mistral-v0.3-7B/tokenizer.json')

# Evaluation

## Phi-3.5-3.8B

In [ ]:
phi_output = pd.read_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/wiki-phi-3.5.csv')
references = phi_output["debiased_sent"].tolist()
predictions = phi_output["model_output"].tolist()

In [ ]:
meteor = evaluate.load('meteor')
results = meteor.compute(predictions=predictions, references=references)
results

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


{'meteor': 0.8993447545685992}

In [ ]:
bleu = evaluate.load('bleu')
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.7949553361187344,
 'precisions': [0.864383509400862,
  0.811305136189838,
  0.7720765183342171,
  0.7375982422040057],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0503945839842967,
 'translation_length': 26221,
 'reference_length': 24963}

In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions, references=references)
results

{'rouge1': 0.8806112857399246,
 'rouge2': 0.8196200415595662,
 'rougeL': 0.875628301398448,
 'rougeLsum': 0.8762516569203029}

In [ ]:
bertscore = evaluate.load('bertscore')
results = bertscore.compute(predictions=predictions, references=references, lang='en')


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sum(results['f1'])/len(results['f1'])

0.9770030836124375

## Llama 3.2 3B

In [ ]:
phi_output = pd.read_csv('/content/drive/Shareddrives/FYP 2024-2025/Phase-2/model-evaluation/wiki-llama-3.2-3b.csv')
references = phi_output["debiased_sent"].tolist()
predictions = phi_output["model_output"].tolist()

In [ ]:
meteor = evaluate.load('meteor')
results = meteor.compute(predictions=predictions, references=references)
results

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


{'meteor': 0.8578768061866434}

In [ ]:
bleu = evaluate.load('bleu')
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.7779932922679367,
 'precisions': [0.8692059653365578,
  0.8029050838968195,
  0.7545226348134684,
  0.7131110711718188],
 'brevity_penalty': 0.9938521079056318,
 'length_ratio': 0.9938709289748828,
 'translation_length': 24810,
 'reference_length': 24963}

In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions, references=references)
results

{'rouge1': 0.8545011484437457,
 'rouge2': 0.785533877665191,
 'rougeL': 0.8481923802465374,
 'rougeLsum': 0.8487151969807633}

In [ ]:
bertscore = evaluate.load('bertscore')
results = bertscore.compute(predictions=predictions, references=references, lang='en')


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sum(results['f1'])/len(results['f1'])

0.9610100983174195

In [ ]:
sum(results['precision'])/len(results['precision'])

0.9656838411596459

In [ ]:
sum(results['recall'])/len(results['recall'])

0.956665567459075